# Agents with LlamaIndex II - 

Sources [1](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/), [2](https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/agents/), [3](https://docs.llamaindex.ai/en/stable/examples/agent/custom_agent/), [4](https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent/), [5](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/agent_runner/), [6](https://medium.com/llamaindex-blog/data-agents-eed797d7972f), [LlamaHub Tools](https://llamahub.ai/?tab=tools)  
    
--- 
### Agents

An "agent" is an automated reasoning and decision engine. It takes in a user input/query and can make internal decisions for executing that query in order to return the correct result. The key agent components can include, but are not limited to:

+ Breaking down a complex question into smaller ones
+ Choosing an external Tool to use + coming up with parameters for calling the Tool
+ Planning out a set of tasks
+ Storing previously completed tasks in a memory module

LlamaIndex provides a comprehensive framework for building agents. This includes the following components:

+ Using agents with tools at a high-level to build agentic RAG and workflow automation use cases **(Previous Notebook)**
+ Low-level components for building and debugging agents
+ Core agent ingredients that can be used as standalone modules: query planning, tool use, and more **(Previous Notebook)**

### Tools

Having proper tool abstractions is at the core of building data agents. Defining a set of Tools is similar to defining any API interface, with the exception that these Tools are meant for agent rather than human use. We allow users to define both a Tool as well as a ToolSpec containing a series of functions under the hood.

When using an agent or LLM with function calling, the tool selected (and the arguments written for that tool) rely strongly on the tool name and description of the tools purpose and arguments. Spending time tuning these parameters can result in larges changes in how the LLM calls these tools.

A Tool implements a very generic interface - simply define __call__ and also return some basic metadata (name, description, function schema).
LlamaIndex offer a few different types of Tools:

+ FunctionTool: A function tool allows users to easily convert any user-defined function into a Tool. It can also auto-infer the function schema. **(Previous Notebook)**
+ QueryEngineTool: A tool that wraps an existing query engine. Note: since our agent abstractions inherit from BaseQueryEngine, these tools can also wrap other agents. **(Previous Notebook)**
+ Community contributed ToolSpecs that define one or more tools around a single service (like Gmail): see [LlamaHub](https://llamahub.ai/)
+ Utility tools for wrapping other tools to handle returning large amounts of data from a tool: see [OnDemandLoaderTool](https://docs.llamaindex.ai/en/stable/examples/tools/OnDemandLoaderTool/)

### Use Cases

The scope of possible use cases for agents is vast and ever-expanding. That said, here are some practical use cases that can deliver immediate value.

+ Agentic RAG: Build a context-augmented research assistant over your data that not only answers simple questions, but complex research tasks. (Previous Notebook)
+ SQL Agent: A subset of the above is a "text-to-SQL assistant" that can interact with a structured database. 
+ Workflow Assistant: Build an agent that can operate over common workflow tools like email, calendar.
+ Coding Assistant: Build an agent that can operate over code.

## Installing Packages

In [2]:
#%pip install -qU openai
#%pip install -qU llama-index
#%pip install -qU llama_hub
#%pip install -qU llama-index-experimental
#%pip install llama-index-agent-openai
#%pip install llama-index-readers-web
#%pip install llama-index-tools-google


#!pip install -q pypdf
#!pip install -q docx2txt

## Importing Packages

In [3]:
import os
import openai

#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

import sys
import shutil
import glob
import logging
from pathlib import Path
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import llama_index
from llama_index.llms.openai import OpenAI

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import SummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.experimental.query_engine import PandasQueryEngine

## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node

## LlamaIndex Templates
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.base.llms.types import ChatMessage, MessageRole

## LlamaIndex Agents
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

import nest_asyncio
nest_asyncio.apply()

In [4]:
import logging

#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Defining Models

In [20]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#model="gpt-3.5-turbo"
model="gpt-4"
#model="gpt-4-turbo"

Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.llm = OpenAI(temperature=0, 
                      model=model, 
                      #max_tokens=512
                      PRESENCE_PENALTY=-2,
                      TOP_P=1,
                     )

llm = Settings.llm

---

## 1- Function Tool  
A function tool is a simple wrapper around any existing function (both sync and async are supported!).  

In [21]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def subtract(x: int, y: int) -> int: 
    """Subtract the second number from the first number."""
    return (x - y)

def multiply(x: int, y: int) -> int: 
    """Multiply one number by the other number."""
    return (x * y)

def uppercase(x: str) -> str: 
    """Return the input string in uppercase."""
    return (x.upper())

add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
uppercase_tool = FunctionTool.from_defaults(fn=uppercase)

In [22]:
response = llm.predict_and_call(
    [add_tool, subtract_tool, multiply_tool, uppercase_tool], 
    "Tell me the output of 3 - 12 ", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: subtract with args: {"x": 3, "y": 12}
=== Function Output ===
-9
-9


---

## 2- [Coding Assistant and Interpreter](https://llamahub.ai/l/tools/llama-index-tools-code-interpreter?from=)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-code-interpreter/examples/code_interpreter.ipynb)

In [23]:
%pip install -q llama-index-tools-code_interpreter

Note: you may need to restart the kernel to use updated packages.


In [31]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.code_interpreter import CodeInterpreterToolSpec

code_spec = CodeInterpreterToolSpec()
tools = code_spec.to_tool_list()
agent = OpenAIAgent.from_tools(tools, verbose=True)

In [32]:
print(agent.chat("Can you help me write some python code to pass to the code_interpreter tool"))

Added user message to memory: Can you help me write some python code to pass to the code_interpreter tool
Sure, I'd be happy to help. What would you like the Python code to do?


In [33]:
print(agent.chat(
        """There is a california_housing_train.csv file in the `../Data` directory (relative path).
                 Can you write and execute code to tell me its columns?"""
    )
)

Added user message to memory: There is a california_housing_train.csv file in the `../Data` directory (relative path).
                 Can you write and execute code to tell me its columns?
=== Calling Function ===
Calling function: code_interpreter with args: {
  "code": "import pandas as pd\n\n# Load the data\nfile_path = '../Data/california_housing_train.csv'\ndata = pd.read_csv(file_path)\n\n# Get the column names\ncolumns = data.columns.tolist()\n\n# Print the column names\ncolumns"
}
Got output: StdOut:
b''
StdErr:
b''

=== Calling Function ===
Calling function: code_interpreter with args: {
  "code": "import pandas as pd\n\n# Load the data\nfile_path = '../Data/california_housing_train.csv'\ndata = pd.read_csv(file_path)\n\n# Get the column names\ncolumns = data.columns.tolist()\n\n# Print the column names\nprint(columns)"
}
Got output: StdOut:
b"['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median

In [34]:
print(agent.chat("In which country are these latitudes and longitudes?"))

Added user message to memory: In which country are these latitudes and longitudes?
The dataset is named `california_housing_train.csv`, which suggests that the latitudes and longitudes are likely from locations within the state of California, United States.


In [35]:
print(agent.chat("Can you make a graph of the median income per total rooms?"))

Added user message to memory: Can you make a graph of the median income per total rooms?
=== Calling Function ===
Calling function: code_interpreter with args: {
  "code": "import matplotlib.pyplot as plt\n\n# Calculate median income per total rooms\ndata['income_per_room'] = data['median_income'] / data['total_rooms']\n\n# Create a plot\nplt.figure(figsize=(10, 6))\nplt.scatter(data['total_rooms'], data['income_per_room'])\nplt.title('Median Income per Total Rooms')\nplt.xlabel('Total Rooms')\nplt.ylabel('Median Income per Room')\n\n# Save the plot\nplt.savefig('income_per_room.png')\n\n# Print a message\n'Plot saved as income_per_room.png'"
}
Got output: StdOut:
b''
StdErr:
b'Traceback (most recent call last):\n  File "<string>", line 4, in <module>\nNameError: name \'data\' is not defined\n'

=== Calling Function ===
Calling function: code_interpreter with args: {
  "code": "import pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load the data\nfile_path = '../Data/california_hous

In [36]:
print(agent.chat("I cant see the plot - can you save it locally with file name `output.png`?"))

Added user message to memory: I cant see the plot - can you save it locally with file name `output.png`?
=== Calling Function ===
Calling function: code_interpreter with args: {
  "code": "import pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load the data\nfile_path = '../Data/california_housing_train.csv'\ndata = pd.read_csv(file_path)\n\n# Calculate median income per total rooms\ndata['income_per_room'] = data['median_income'] / data['total_rooms']\n\n# Create a plot\nplt.figure(figsize=(10, 6))\nplt.scatter(data['total_rooms'], data['income_per_room'])\nplt.title('Median Income per Total Rooms')\nplt.xlabel('Total Rooms')\nplt.ylabel('Median Income per Room')\n\n# Save the plot\nplt.savefig('output.png')\n\n# Print a message\nprint('Plot saved as output.png')"
}
Got output: StdOut:
b'Plot saved as output.png\n'
StdErr:
b''

The plot of Median Income per Total Rooms has been created and saved as 'output.png'. Please check the file in your local directory.


---

## 3- [SQL Agent](https://llamahub.ai/l/tools/llama-index-tools-database?from=)  
[GitHub](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-database/examples/database.ipynb), [article](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo/)

In [70]:
%pip install -q llama-index-tools-database
%pip install -q sqlalchemy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [38]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.database import DatabaseToolSpec
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    insert,
    text,
)

In [39]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [40]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

#### Define SQL Database   

We first define our SQLDatabase abstraction (a light wrapper around SQLAlchemy).

In [41]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

#### We add some testing data to our SQL database

In [42]:
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States",},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [43]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


#### Query Index  
We first show how we can execute a raw SQL query, which directly executes over the table.  

In [44]:
with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


#### Part 1: Text-to-SQL Query Engine¶

Once we have constructed our SQL database, we can use the NLSQLTableQueryEngine to construct natural language queries that are synthesized into SQL queries.

Note that we need to specify the tables we want to use with this query engine. If we don't the query engine will pull all the schema context, which could overflow the context window of the LLM.

This query engine should be used in any case where you can specify the tables you want to query over beforehand, or the total size of all the table schema plus the rest of the prompt fits your context window.

In [45]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [46]:
from IPython.display import Markdown
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of 13,960,000.</b>

#### Part 2: Query-Time Retrieval of Tables for Text-to-SQL

If we don't know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we should store the table schema in an index so that during query time we can retrieve the right schema.

The way we can do this is using the SQLTableNodeMapping object, which takes in a SQLDatabase and produces a Node object for each SQLTableSchema object passed into the ObjectIndex constructor.  

In [47]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
#from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

#### Now we can take our SQLTableRetrieverQueryEngine and query it for our response.

In [48]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of 13,960,000.</b>

In [49]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[('Tokyo', 13960000)]

#### You could also add additional context information for each table schema you define.

In [50]:
# manually set context text
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text))
]

#### Part 3: Text-to-SQL Retriever

So far our text-to-SQL capability is packaged in a query engine and consists of both retrieval and synthesis.

You can use the SQL retriever on its own. We show you some different parameters you can try, and also show how to plug it into our RetrieverQueryEngine to get roughly the same results.


In [51]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True
)

In [52]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [53]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** a81d44d2-d88b-44bd-9746-66207d36cc6d<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>

In [54]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

In [55]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [56]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 74b280e8-be46-4b94-8306-c38ed4f21e9b<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** 81f913d9-5b29-42db-832b-2f5c3af96cb0<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** 8775e81d-d11c-4415-9fb8-b93bfdc8eaae<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** d3f2acb6-f546-4f17-83f5-5869bcc00bd6<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

#### Plug into our RetrieverQueryEngine

We compose our SQL Retriever with our standard RetrieverQueryEngine to synthesize a response. The result is roughly similar to our packaged Text-to-SQL query engines.

In [57]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [58]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [59]:
print(str(response))

1. Tokyo - 13,960,000
2. Seoul - 9,776,000
3. Toronto - 2,930,000
4. Chicago - 2,679,000


#### Using Database Tool

This tool connects to a database (using SQLAlchemy under the hood) and allows an Agent to query the database and get information about the tables.

In [60]:
db_tools = DatabaseToolSpec(sql_database)
agent = OpenAIAgent.from_tools(db_tools.to_tool_list())

agent.chat("What tables does this database contain").response

"This database contains a table named 'city_stats'."

In [61]:
print(agent.chat("Describe the first table").response)

The 'city_stats' table has the following structure:

- `city_name`: A string (VARCHAR) of maximum 16 characters. This is the primary key and cannot be null.
- `population`: An integer representing the population of the city.
- `country`: A string (VARCHAR) of maximum 16 characters. This cannot be null.


In [62]:
print(agent.chat("Retrieve the first row of that table").response)

The first row of the 'city_stats' table is:

- City Name: Toronto
- Population: 2,930,000
- Country: Canada


---

## 4- Community Tools

### A) [Wikipedia Tool](https://llamahub.ai/l/tools/llama-index-tools-wikipedia?from=tools)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-wikipedia/examples/wikipedia.ipynb)

In [71]:
%pip install -q llama-index-tools-wikipedia

Note: you may need to restart the kernel to use updated packages.


In [64]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.wikipedia import WikipediaToolSpec

tool_spec = WikipediaToolSpec()

agent = OpenAIAgent.from_tools(tool_spec.to_tool_list()) #, verbose=True,)

response= agent.chat("Who is Ben Afflecks spouse?")

#print(response.sources)
print(response.response)

Ben Affleck's current spouse is Jennifer Lopez. They got back together in 2021 after previously being engaged from 2002 to 2004. Before that, he was married to actress Jennifer Garner from 2005 to 2018.


### B) [Arxiv Tool](https://llamahub.ai/l/tools/llama-index-tools-arxiv?from=tools)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-arxiv)  

In [72]:
%pip install -q llama-index-tools-arxiv

Note: you may need to restart the kernel to use updated packages.


In [67]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.arxiv.base import ArxivToolSpec

arxiv_tool = ArxivToolSpec()

agent = OpenAIAgent.from_tools(arxiv_tool.to_tool_list(), verbose=True,)

print(agent.chat("Whats are the newest discoveries on Prompt Engineering?"))

Added user message to memory: Whats are the newest discoveries on Prompt Engineering?
=== Calling Function ===
Calling function: arxiv_query with args: {
  "query": "Prompt Engineering",
  "sort_by": "recent"
}
Got output: [Document(id_='8d38e7e5-66a6-4d0f-a3ea-cc1531d30f0e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='http://arxiv.org/pdf/2405.19325v1: Nearest Neighbor Speculative Decoding for LLM Generation and Attribution\nLarge language models (LLMs) often hallucinate and lack the ability to\nprovide attribution for their generations. Semi-parametric LMs, such as kNN-LM,\napproach these limitations by refining the output of an LM for a given prompt\nusing its nearest neighbor matches in a non-parametric data store. However,\nthese models often exhibit slow inference speeds and produce non-fluent texts.\nIn this paper, we introduce Nearest Neighbor Speculative Decoding (NEST), a\nnovel semi-parametric language 

### C) [DuckDuckGoSearch Tool](https://llamahub.ai/l/tools/llama-index-tools-duckduckgo?from=tools)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-duckduckgo/examples/duckduckgo_search.ipynb)  

In [69]:
%pip install -q llama-index-tools-duckduckgo

Note: you may need to restart the kernel to use updated packages.


In [73]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

tool_spec = DuckDuckGoSearchToolSpec()

agent = OpenAIAgent.from_tools(DuckDuckGoSearchToolSpec().to_tool_list(), verbose=True,)
response = agent.chat("what are the latest developments in machine learning")

Added user message to memory: what are the latest developments in machine learning
=== Calling Function ===
Calling function: duckduckgo_full_search with args: {
  "query": "latest developments in machine learning",
  "max_results": 5
}
Got output: [{'title': 'Machine learning | MIT News | Massachusetts Institute of Technology', 'href': 'https://news.mit.edu/topic/machine-learning', 'body': "Machine learning. Download RSS feed: News Articles / In the Media / Audio. Displaying 1 - 15 of 900 news articles related to this topic. ... A new algorithm learns to squish, bend, or stretch a robot's entire body to accomplish diverse tasks like avoiding obstacles or retrieving items."}, {'title': '10 top AI and machine learning trends for 2024 | TechTarget', 'href': 'https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends', 'body': "This year's trends reflect a deepening sophistication and caution in AI development and deployment strategies, with an eye to ethics, s

In [74]:
print(response.response)

Here are some of the latest developments in machine learning:

1. [MIT News](https://news.mit.edu/topic/machine-learning) reports on a new algorithm that learns to manipulate a robot's entire body to accomplish diverse tasks like avoiding obstacles or retrieving items.

2. [TechTarget](https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends) highlights the top 10 AI and machine learning trends for 2024, including Multimodal AI, which goes beyond traditional single-mode data processing.

3. [Waleed's Blog](https://waleed.dev/blog/latest-developments-in-machine-learning-2024-update) mentions some of the latest developments shaping the industry in 2024, such as Generative AI. Models like DALL-E 3, Stable Diffusion, and Google's Imagen are producing highly realistic synthetic images and videos, impacting content creation, entertainment, art, and more.

4. [HackerRank Blog](https://www.hackerrank.com/blog/top-machine-learning-trends/) discusses seven trends wo

In [75]:
agent.chat_history.clear()
print(agent.chat("Who is Goethe?"))

Added user message to memory: Who is Goethe?
=== Calling Function ===
Calling function: duckduckgo_instant_search with args: {
  "query": "Goethe"
}
Got output: [{'icon': None, 'text': 'Johann Wolfgang von Goethe was a German polymath and writer, who is widely regarded as the greatest and most influential writer in the German language. His work has had a profound and wide-ranging influence on Western literary, political, and philosophical thought from the late 18th century to the present day. Goethe was a German poet, playwright, novelist, scientist, statesman, theatre director, and critic. His works include plays, poetry and aesthetic criticism, as well as treatises on botany, anatomy, and color. Goethe took up residence in Weimar in November 1775 following the success of his first novel, The Sorrows of Young Werther. He was ennobled by the Duke of Saxe-Weimar, Karl August, in 1782. Goethe was an early participant in the Sturm und Drang literary movement.', 'topic': None, 'url': 'http

### D) [Yahoo Finance Tool](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/tools/)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-yahoo-finance)  

In [76]:
%pip install -q llama-index-tools-yahoo-finance

Note: you may need to restart the kernel to use updated packages.


In [77]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

tool_spec = YahooFinanceToolSpec()
agent = OpenAIAgent.from_tools(tool_spec.to_tool_list())

print(agent.chat("What is the price of Apple stock?").response)

The current price of Apple Inc. (AAPL) stock is $191.865.


In [78]:
print(agent.chat("What is the latest news about Apple?").response)

Here are the latest news headlines about Apple:

1. "Magnificent Seven Stocks: Nvidia Stock Reverses From Record Highs"
2. "Apple iPhone Sales Weak In China, U.S.: Report"
3. "Warner Bros. Discovery CEO says company has 'full buffet' of sports as NBA rights decision looms"
4. "Apple Stock Isn’t a Buy Yet, Says Analyst. China-iPhone Demand Remains an Issue."
5. "Fighting Off Nvidia, This Mag 7 Stock Looks To Break Out"
6. "Apple Expected to Accelerate Growth on Strength of Investment in AI, Tigress Financial Partners Says"
7. "Dow Jones Futures Fall As Salesforce Plunges After Stock Market Slides Despite Nvidia"
8. "The Most Prosperous Nation in the World"


### E) [Open Weather Map Tool](https://llamahub.ai/l/tools/llama-index-tools-weather?from=tools)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-weather)  

In [79]:
%pip install -q llama-index-tools-weather
%pip install -q pyowm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [80]:
#from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.weather import OpenWeatherMapToolSpec

os.environ["OPENWEATHER_API_KEY"]="your key"

## https://openweathermap.org/api
tool_spec = OpenWeatherMapToolSpec(key=os.environ["OPENWEATHER_API_KEY"])

agent = OpenAIAgent.from_tools(tool_spec.to_tool_list())

agent.chat("What is the temperature like in Paris?")

AgentChatResponse(response="In Paris, the current weather is as follows:\n- Detailed status: moderate rain\n- Wind speed: 7.2 m/s, direction: 290°\n- Humidity: 77%\n- Temperature: \n  - Current: 14.88°C\n  - High: 16.64°C\n  - Low: 12.78°C\n  - Feels like: 14.43°C\n- Rain: {'1h': 0.12}\n- Heat index: None\n- Cloud cover: 75%", sources=[ToolOutput(content='[Document(id_=\'09ac82fa-8c96-472c-bac3-31727d21cc3b\', embedding=None, metadata={\'weather from\': \'Paris\'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="In Paris, the current weather is as follows:\\nDetailed status: moderate rain\\nWind speed: 7.2 m/s, direction: 290°\\nHumidity: 77%\\nTemperature: \\n  - Current: 14.88°C\\n  - High: 16.64°C\\n  - Low: 12.78°C\\n  - Feels like: 14.43°C\\nRain: {\'1h\': 0.12}\\nHeat index: None\\nCloud cover: 75%", start_char_idx=None, end_char_idx=None, text_template=\'{metadata_str}\\n\\n{content}\', metadata_template=\'{key}: {value}\', metadata_sepera

In [85]:
agent.chat("What is the temperature in Vienna today?")

AgentChatResponse(response="In Vienna, the current weather is as follows:\n- Detailed status: light rain\n- Wind speed: 2.57 m/s, direction: 210°\n- Humidity: 86%\n- Temperature: \n  - Current: 17.13°C\n  - High: 18.93°C\n  - Low: 15.64°C\n  - Feels like: 17.14°C\n- Rain: {'1h': 0.23}\n- Heat index: None\n- Cloud cover: 20%", sources=[ToolOutput(content='[Document(id_=\'5153a3c3-5dad-4193-b3e5-15d3e0bc10a6\', embedding=None, metadata={\'weather from\': \'Vienna\'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="In Vienna, the current weather is as follows:\\nDetailed status: light rain\\nWind speed: 2.57 m/s, direction: 210°\\nHumidity: 86%\\nTemperature: \\n  - Current: 17.13°C\\n  - High: 18.93°C\\n  - Low: 15.64°C\\n  - Feels like: 17.14°C\\nRain: {\'1h\': 0.23}\\nHeat index: None\\nCloud cover: 20%", start_char_idx=None, end_char_idx=None, text_template=\'{metadata_str}\\n\\n{content}\', metadata_template=\'{key}: {value}\', metadata_seperat

## 5- [OnDemandLoaderTool](https://docs.llamaindex.ai/en/stable/examples/tools/OnDemandLoaderTool/)  

OnDemandLoaderTool is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.  

This tool takes in a BaseReader data loader, and when called will 1) load data, 2) index data, and 3) query the data.  

In this walkthrough, we show how to use the OnDemandLoaderTool to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [86]:
%pip install -q llama-index-readers-wikipedia

Note: you may need to restart the kernel to use updated packages.


In [87]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

#### Define Tool¶

First it is defined the WikipediaReader. Note that the load_data interface to WikipediaReader takes in a list of pages.   
By default, this queries the Wikipedia search endpoint which will autosuggest the relevant pages.  
Then it wraps it into our OnDemandLoaderTool.  
By default since we don't specify the index_cls, a simple vector store index is initialized.  

In [88]:
reader = WikipediaReader()

tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

# run tool by itself
tool(["Berlin"], query_str="What's the arts and culture scene in Berlin?")

ToolOutput(content="Berlin has a rich and diverse arts and culture scene. The city is known for its vibrant nightlife, with a history of being a hub for punk music and culture, and a premier nightlife venue for people from all over the world. It has many nightclubs, and is known for its uninhibited parties. Berlin also has a long history of gay culture and hosts a number of queer clubs and festivals.\n\nThe city is home to 44 theaters and stages, including the Deutsches Theater, the Volksbühne, and the Berliner Ensemble. It also has three major opera houses: the Deutsche Oper, the Berlin State Opera, and the Komische Oper. The city's main venue for musical theater performances are the Theater am Potsdamer Platz and Theater des Westens. Contemporary dance can be seen at the Radialsystem V.\n\nBerlin also hosts the Berlin International Film Festival, which is considered to be the largest publicly attended film festival in the world. Other cultural festivals include the Karneval der Kultu

In [89]:
#from llama_index.indices.vector_store import VectorStoreIndex

reader = WikipediaReader()
tool = OnDemandLoaderTool.from_defaults(
    reader=reader,
    index_cls=VectorStoreIndex,
    index_kwargs={"dim": 768},
    name="Wikipedia",
    description="A tool for querying Wikipedia articles.",
)

In [90]:
response = tool(["City of Vienna"], query_str="What are the best universities in Vienna?")

In [91]:
response.content

'The context information lists several universities in Vienna, including the Academy of Fine Arts Vienna, Central European University, Diplomatic Academy of Vienna, Medical University of Vienna, PEF Private University of Management Vienna, University of Applied Arts Vienna, University of Applied Sciences Campus Vienna, University of Music and Performing Arts Vienna, University of Veterinary Medicine Vienna, University of Vienna, Vienna University of Economics and Business, University of Natural Resources and Life Sciences, Vienna, University of Applied Sciences Technikum Wien, TU Wien, Webster Vienna Private University, and Sigmund Freud Private University. However, it does not rank them or specify which ones are considered the best.'

## Also check [These Notebooks](https://github.com/run-llama/llama-hub/tree/main/llama_hub/tools/notebooks)